In [1]:
# Instalação das dependências para a parte prática

!pip install pydantic fastapi uvicorn

In [2]:
# Importação das dependências

from uuid import uuid4, UUID
from datetime import date
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field, field_validator
from fastapi.testclient import TestClient

# Exemplo utilizando Pydantic e FastAPI para fazer uma API de "to do list"


A ideia do código é implementar uma API de tarefas (to do list) utilizando FastAPI e Pydantic. Ele define um modelo ToDoItem com validação de data de vencimento, gera IDs automaticamente e controla um campo de conclusão.

In [4]:
# Cria instância do aplicativo FastAPI
app = FastAPI()

# Lista em memória para armazenar as tarefas criadas
todos: List["ToDoItem"] = []

class ToDoItem(BaseModel):
    # Modelo de dado para cada tarefa da lista
    id: UUID = Field(
        default_factory=uuid4,
        description="Identificador único gerado automaticamente"
    )
    title: str = Field(..., min_length=1, description="Título da tarefa, mínimo 1 caractere")
    description: Optional[str] = Field(
        default=None,
        description="Descrição opcional da tarefa"
    )
    due_date: Optional[date] = Field(
        default=None,
        description="Data de vencimento opcional"
    )
    completed: bool = Field(
        default=False,
        description="Flag que indica se a tarefa já foi concluída"
    )

    @field_validator("due_date", mode="before")
    @classmethod
    def check_due_date_future(cls, v): # Valida que a data de vencimento não seja anterior à data de hoje
        if v is not None and v < date.today():
            raise ValueError("due_date não pode ser no passado")
        return v

@app.post("/todos", response_model=ToDoItem)
def create_todo(item: ToDoItem): # Recebe um objeto ToDoItem válido, adiciona na lista e retorna-o
    todos.append(item)
    return item

@app.get("/todos", response_model=List[ToDoItem])
def list_todos(): # Retorna a lista completa de tarefas em memória
    return todos

@app.patch("/todos/{item_id}/complete", response_model=ToDoItem)
def complete_todo(item_id: UUID): # Procura tarefa pelo ID e marca como concluída
    for todo in todos:
        if todo.id == item_id:
            todo.completed = True
            return todo
    raise HTTPException(status_code=404, detail="Tarefa não encontrada") # Se não encontrar, retorna erro 404

In [5]:
# Instancia o cliente de testes do FastAPI para verificar funcionalidades
client = TestClient(app)

# 1) Criar uma tarefa sem data de vencimento
resp1 = client.post(
    "/todos",
    json={"title": "Comprar leite"}
)
print("POST /todos:", resp1.status_code, resp1.json())

# 2) Listar todas as tarefas criadas
resp2 = client.get("/todos")
print("GET /todos:", resp2.status_code, resp2.json())

# 3) Marcar a primeira tarefa como concluída usando seu ID retornado
tid = resp2.json()[0]["id"]
resp3 = client.patch(f"/todos/{tid}/complete")
print(f"PATCH /todos/{tid}/complete:", resp3.status_code, resp3.json())

POST /todos: 200 {'id': 'de1b4960-f576-4543-b10d-28bf644b9640', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': False}
GET /todos: 200 [{'id': 'de1b4960-f576-4543-b10d-28bf644b9640', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': False}]
PATCH /todos/de1b4960-f576-4543-b10d-28bf644b9640/complete: 200 {'id': 'de1b4960-f576-4543-b10d-28bf644b9640', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': True}
